In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, plot_confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold
from sklearn.pipeline import make_pipeline

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

from pandas.plotting import table 

In [2]:
plt.rcParams['font.size'] = 8
plt.rc('font', size=10) 

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
encoded_df = pd.read_pickle('../data/encoded_df.pkl')
encoded_df.head()

,I identify as having a mental illness,Education,I have my own computer separate from a smart phone,I have been hospitalized before for my mental illness,How many days were you hospitalized for your mental illness,I am legally disabled,I have my regular access to the internet,I live with my parents,I have a gap in my resume,Total gaps in resume in months,Annual income (including any social welfare programs) in USD,I am unemployed,I read outside of work and school,Annual income from social welfare programs,I receive food stamps,I am on section 8 housing,How many times were you hospitalized for your mental illness,Lack of concentration,Anxiety,Depression,Obsessive_thinking,Mood_swings,Panic_attacks,Compulsive_behavior,Tiredness,Age,Gender,Household Income,Region_East North Central,Region_East South Central,Region_Middle Atlantic,Region_Mountain,Region_New England,Region_Pacific,Region_South Atlantic,Region_West North Central,Region_West South Central,Device Type_Android Phone / Tablet,Device Type_MacOS Desktop / Laptop,Device Type_Other,Device Type_Windows Desktop / Laptop,Device Type_iOS Phone / Tablet,time_to_complete
0,0,1,0,0,0,0,1,0,1,24,35,1,1,0,0,0,0,1,1,1,1,0,1,0,0,1,0,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,188.0
1,1,5,1,0,0,0,1,0,0,1,22,0,1,0,0,0,0,1,1,1,0,0,1,0,1,0,0,4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,65.0
2,0,3,1,0,0,0,1,0,0,0,100,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,8,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,141.0
3,0,2,1,0,0,0,1,1,1,11,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,77.0
4,1,3,1,1,35,1,1,0,1,33,32,0,1,30,0,0,4,1,1,1,1,1,1,1,1,1,0,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,142.0


### Establish Numeric Columns for Scaling
- excluding binary cols and categorical cols that have been encoded

In [4]:
features_to_scale = ['How many days were you hospitalized for your mental illness',
                     'Total gaps in resume in months',
                     'Annual income (including any social welfare programs) in USD',
                     'Annual income from social welfare programs',
                     'How many times were you hospitalized for your mental illness',
                     'time_to_complete'
                    ]

### Function for properly scaling the numerical data

In [5]:
# A good replacement for this function would be to use the ColumnTransformer with a Pipeline
# Though since this data needed such complicated cleaning, it seemed appropriate to handle this way
def scale_and_merge(traing_data, test_data, cols_to_scale):
    scaler = StandardScaler()
    train_scaled_features = scaler.fit_transform(traing_data[cols_to_scale])
    train_scaled_features_df = pd.DataFrame(train_scaled_features, columns=cols_to_scale)
    
    training_scaled_df = traing_data.drop(features_to_scale, axis=1)
    training_scaled_df = pd.merge(training_scaled_df, train_scaled_features_df, right_index=True, left_index=True)
    
    test_scaled_features = scaler.transform(test_data[cols_to_scale])
    test_scaled_features_df = pd.DataFrame(test_scaled_features, columns=cols_to_scale)
    
    test_scaled_df = test_data.drop(features_to_scale, axis=1)
    test_scaled_df = pd.merge(test_scaled_df, test_scaled_features_df, right_index=True, left_index=True)
    
    return training_scaled_df, test_scaled_df

# Modeling

## Question: Can we predict unemployment from mental health data?
### Target: 'I am unemployed'

#### V1 - using all features in cleaned df

###### Notes:
Stratify the train-test split to ensure even distribution of classes between the splits

In [6]:
random_seed = 13

y = encoded_df['I am unemployed']
X = encoded_df.drop('I am unemployed', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state=random_seed, stratify=y)

In [7]:
def score_fitted_model(model, X_test, y_test, file_name=None, title=None):
    target_labels = ['Employed', 'Unemployed']
    
    y_hat = model.predict(X_test)
    
    score_types = {'Accuracy':accuracy_score,'Precision':precision_score, 'Recall':recall_score, 'F1':f1_score}
    
    for metric_name, metric_func in score_types.items():
        print(f'{metric_name}: {round(metric_func(y_test, y_hat),2)}')
    
    fig, ax = plt.subplots(figsize=(4,4))
    if title: 
        ax.set_title(title)
        
    plot_confusion_matrix(model, X_test, y_test, display_labels=target_labels, ax=ax)
    if file_name:
        plt.savefig(f'../img/{file_name}.png', bbox_inches="tight")
    plt.show()
    
    #print(classification_report(y_true=y_test, y_pred=y_hat, target_names=target_labels))

### Logistic Regression Model

In [ ]:
lr = LogisticRegression()
scaler = StandardScaler()
X_train_scaled  = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr.fit(X_train_scaled, y_train)

score_fitted_model(lr, X_test_scaled, y_test, file_name='conf_mat_baseline_log_reg', title='Baseline Logistic Regression')

In [ ]:
X_test_scaled[0:5]

### RandomForest

In [ ]:
rf_clf = RandomForestClassifier(random_state=random_seed)
rf_clf.fit(X_train, y_train)

score_fitted_model(rf_clf, X_test, y_test, file_name='conf_mat_baseline_rand_for', title='Baseline Random Forest')

The top 20 features

In [ ]:
pd.Series(index=X.columns, data=rf_clf.feature_importances_).sort_values(ascending=False)[:10].round(3)

In [ ]:
def print_pd_table(save_df, fsize, filename):
    fig, ax = plt.subplots(figsize=(8,3)) # set size frame
    ax.xaxis.set_visible(False)  # hide the x axis
    ax.yaxis.set_visible(False)  # hide the y axis
    ax.set_frame_on(False)  # no visible frame, uncomment if size is ok
    tabla = table(ax, save_df, loc='upper right', colWidths=[.2]*len(save_df.columns))
    tabla.auto_set_font_size(False) # Activate set fontsize manually
    tabla.set_fontsize(12) # if ++fontsize is necessary ++colWidths
    tabla.scale(1.2, 1.2) # change size table
    #plt.savefig('table.png', transparent=True)
    plt.tight_layout()
    plt.savefig(filename, transparent=True)

In [ ]:
save_df = pd.DataFrame(index=X.columns, data=rf_clf.feature_importances_, columns=['importance']).sort_values(by='importance', ascending=False)[:10].round(3)
print_pd_table(save_df, (8,3), '../img/baseline_feature_importance2.png')

In [ ]:
save_df

#### Interpreting results.
##### Accuracy and precision are pretty good, but the recall score is not. 
What is the best metric for this case? The cost of a false positive, i.e. predicting someone will become unemployed but they don't, is not likely a great concern, so precision is going to be primary.

On the other hand, a false negative of predicing a person won't become unemployed but does, would mostly likely be a case we would want to avoid the most. Therefore, recall is more important than precision or accuracy.

F1 is a balance of recall and precision. As expected it is somewhere inbetween the two, but it is most helpful in situations where there is a class imbalance and there are more actual negatives. This is the case with our data where we have about a 3:1 ratio of negative to positive target responses. So the F1 score may be the best score for our case.

First, let's use cross validation to see if these results hold up.

#### V2 - Hyperparameter Tuning with All Features in Data

In [ ]:
def tune_model(X, y, pipe, params):
    gridsearch = GridSearchCV(estimator=pipe, param_grid=params, scoring='f1', verbose=2, n_jobs=-1).fit(X, y)
    print(f'Best F1 Score: {gridsearch.score(X, y)}')
    
    best_params = gridsearch.best_params_
    print(best_params)
    
    return gridsearch    

In [ ]:
pipe_lr = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('estimator', LogisticRegression(max_iter=600, random_state=random_seed))
])

# define parameter ranges in dict
# use double underscore to link pipline object with param name -
# - use the label created when defining the pipe for the test left of the '__'
params_lr = {
    'estimator__solver' : ['lbfgs','liblinear', 'saga'],
    'estimator__penalty' : ['l1','l2','elasticnet'],
    'estimator__class_weight' : ['balanced', None]
}

gridpipe_lr = tune_model(X_train_scaled, y_train, pipe_lr, params_lr)

In [ ]:
score_fitted_model(gridpipe_lr, X_test_scaled, y_test, file_name=None, title='Logistic Regression Gridsearch')

In [ ]:
pipe_rf = Pipeline(steps=[
    ## RandomForests/Decision Trees don't benefit from scaling('scaler', StandardScaler()),
    ('estimator', RandomForestClassifier(random_state=random_seed))
])

# define parameter ranges in dict
# use double underscore to link pipline object with param name -
# - use the label created when defining the pipe for the test left of the '__'
params_rf = {
    'estimator__n_estimators' : np.arange(40, 111, 10),
    'estimator__max_depth' : np.arange(8, 20, 1),
    'estimator__max_features' : ['auto', 'sqrt', 'log2']
}

gridpipe_rf = tune_model(X_train, y_train, pipe_rf, params_rf)

In [ ]:
score_fitted_model(gridpipe_rf, X_test, y_test, title='Random Forest Gridsearch')

## Results So Far
The logistic regression is outperforming the RandomForest, which appears to be overfitting. Gridsearch improved recal for the Logistic Regression model by .12 and did not help improve the Random Forest.

Let's remove all income data because of possible colinearity.
And date data, device type and hospitalizations because of low correlation with target

In [ ]:
drop_me = [
           'Annual income (including any social welfare programs) in USD',
           'Annual income from social welfare programs',
           'I have been hospitalized before for my mental illness', 
           'How many days were you hospitalized for your mental illness',
           'I have a gap in my resume'
          ]

In [ ]:
X_reduced = X.drop(columns=drop_me)

In [ ]:
X_train_red, X_test_red, y_train_red, y_test_red = train_test_split(X_reduced, y, test_size = .2, random_state=random_seed, stratify=y)

In [ ]:
lr2 = LogisticRegression(random_state=random_seed)
scaler = StandardScaler()
X_train_red_scaled  = scaler.fit_transform(X_train_red)
X_test_red_scaled = scaler.transform(X_test_red)

lr2.fit(X_train_red_scaled, y_train_red)

score_fitted_model(lr2, X_test_red_scaled, y_test_red, title='Logistic Regression X Reduced')

In [ ]:
gridpipe_lr2 = tune_model(X_train_red_scaled, y_train_red, pipe_lr, params_lr)
score_fitted_model(gridpipe_lr2, X_test_red_scaled, y_test_red, title='Logistic Regression Grid Search X Reduced')

## V3 - Using Recusive Feature Elimination To Find Best Features

In [ ]:
def plot_best_num_of_features(model, rfecv, rfe_scoring, base_filename):
    plt.figure(figsize=(8,4))
    plt.title(f'{model.__class__.__name__} Recusive Feature Elimination with CV Results' )
    plt.xlabel('Num of features selected', fontsize=12, labelpad=20)
    plt.ylabel(f'{rfe_scoring.capitalize()} Score',fontsize=12, labelpad=20)
    plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_, linewidth=3)
    plt.savefig(f'../img/{base_filename}_rfe_num_features.png', bbox_inches='tight')
    plt.show()
    
    print('Optimal number of features: {}'.format(rfecv.n_features_))

In [ ]:
def get_top_rfe_features(X, rfecv):
    M = np.array([X.columns, rfecv.ranking_])
    df_rfe = pd.DataFrame(data=M.T, columns=['feature', 'ranking'])
    
    top_features = df_rfe[df_rfe['ranking']==1]
    display(top_features)
    
    X_most_imp = X[top_features['feature']].copy()
    
    return X_most_imp

In [ ]:
def run_rfe(X, y, model, base_filename, rfe_scoring='f1', scale_features=False):
    rfecv = RFECV(estimator=model, step=1, cv=StratifiedKFold(10), scoring=rfe_scoring, n_jobs=-1)
    
    # this step is undesireable - scaling the entire X matrix will create data leakage - but I can't figure out how to scale within the RFECV process
    if scale_features:
        X2 = StandardScaler().fit_transform(X)
        rfecv.fit(X2, y)
    else:
        X2 = X.copy()
        
    rfecv.fit(X2, y)
    
    plot_best_num_of_features(model, rfecv, rfe_scoring, base_filename)
        
    # Here we use the original X data so that proper scaling can occur if needed
    X_most_imp = get_top_rfe_features(X, rfecv)
    
    X_train_rfe, X_test_rfe, y_train_rfe, y_test_rfe = train_test_split(X_most_imp, y, test_size = .2, random_state=random_seed, stratify=y)

    if scale_features:
        scaler_rfe = StandardScaler()
        X_train_rfe  = scaler_rfe.fit_transform(X_train_rfe)
        X_test_rfe = scaler_rfe.transform(X_test_rfe)
        
    model.fit(X_train_rfe, y_train_rfe)
    print('Re-run the model with only the most important features of the same train-test split data used in the previous modeling so we can accurately compare the results.')
    print('')
    score_fitted_model(model, X_test_rfe, y_test_rfe, file_name=f'{base_filename}_conf_mat_rfe', title=f'{model.__class__.__name__} with RFECV')
    
    return X_most_imp.columns

### RFECV with Random Forest

In [ ]:
X_most_imp_rand_for = run_rfe(X, y, RandomForestClassifier(random_state=random_seed), base_filename='rand_forest')

#### Notice the engineered feature, "time_to_complete" is one of the most important

### RFECV with Logistic Regression

In [ ]:
X_most_imp_log_reg = run_rfe(X, y, LogisticRegression(random_state=random_seed), base_filename='logistic_regression', scale_features=True)

### Does Hyperparameter tuning with Gridsearch help the Random Forest?

In [ ]:
X_most_imp = X[X_most_imp_rand_for].copy()

In [ ]:
# recreating the train test split from the RFECV function
X_train_rfe, X_test_rfe, y_train_rfe, y_test_rfe = train_test_split(X_most_imp, y, test_size = .2, random_state=random_seed, stratify=y)

In [ ]:
pipe_rf_rfe = Pipeline(steps=[
    ## RandomForests/Decision Trees don't benefit from scaling('scaler', StandardScaler()),
    ('estimator', RandomForestClassifier(random_state=random_seed))
])

# define parameter ranges in dict
# use double underscore to link pipline object with param name -
# - use the label created when defining the pipe for the test left of the '__'
params_rf_rfe = {
    'estimator__n_estimators' : np.arange(40, 111, 10),
    'estimator__max_depth' : np.arange(8, 20, 1),
    'estimator__max_features' : ['auto', 'sqrt', 'log2']
}

gridpipe_rf_rfe = tune_model(X_train_rfe, y_train_rfe, pipe_rf_rfe, params_rf_rfe)

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
score_fitted_model(gridpipe_rf_rfe, X_test_rfe, y_test_rfe, file_name='conf_mat_rfe_gridsearch')

## No, it doesn't improve things

In [ ]:
def run_cv_rf(X, y):
    pipe = make_pipeline(RandomForestClassifier())
    skf = StratifiedKFold(n_splits=10, random_state=11, shuffle=True)
    scores = cross_val_score(pipe, X, y, cv=skf, scoring='accuracy')
    print(f'Accuracy Scores: {scores}')
    print(f'\tMean:{scores.mean()}')
    scores = cross_val_score(pipe, X, y, cv=skf, scoring='recall')
    print(f'Recall Scores: {scores}')
    print(f'\tMean:{scores.mean()}')
    scores = cross_val_score(pipe, X, y, cv=skf, scoring='f1')
    print(f'F1 Scores: {scores}')
    print(f'\tMean:{scores.mean()}')

run_cv_rf(X, y)

# Possible Additional Steps
- PCA?
- Oversampling to balance classes?
- Turn buckets into averages instead of OneHotEncoding?